# <center>Testing News Asymmetry Between Small and Large Companies</center>


### <center>A sentiment analysis study by Robert Grote and Ryan Fairhurst</center>

This is an interactive Python notebook, which requires that the user run the cells within in order to see the results. Run the cells from the top, all the way down the page by clicking on the first cell, then holding down shift or command or control (depending on whether you're on PC or Mac), then hitting enter.

Install the modules that you will need for reading in all of the data by running the following cell.    
Uncomment the following two lines the first time you run this in order to install the necessary modules by deleting the hash symbol.

In [ ]:
!pip install pandas-datareader
!pip install yahoo-finance
!pip install matplotlib

Import the following modules:

In [ ]:
import yahoo_finance as yfin
import pandas as pd
import pandas_datareader.data as web
import datetime
import csv
import math
import pandas_datareader.data as data
from yahoo_finance import Share
from pandas_datareader.yahoo.quotes import _yahoo_codes
import matplotlib

The basic idea is that, as companies grow in size, the relative proportion of bad news grows with respect to the amount of good news. We are interested in testing whether this is true and appears in the data, whether it differs by industry, whether there is a maximum threshold of how much bad news can be experienced by small companies, and how the proportions change by company size. In the following  prototype, we conceptualize using jumps in Google Trends data and jumps in stock price data to detect newsworthy events, and observations which will amount to the idea of linking sentiment to a company, then recording its size and several other key variables that we think will have interesting relationships with company size and sentiment.

In [ ]:
#'Jumps' in data--a place for our ideas
"""
Defining a Jump
When an event happens
Positive/Negative Event
Positive/Negative interest
"""
#trends-->prices
#prices-->trends
#volume-->prices


#Jump Program: Percentage change--not dollar change
#Different times (Stamp when, magnitude, duration, direction)
#--make a program that measures relatively, not absolutely (percentages, not dollars)
#Sustained vs transient growth

#Jumps:
#1: Identify Jumps
#2:Categorize comparable categories
##Compare news profiles (trends)
##Compare

#Var Types:
#C-Continuous
#S-Scalar
#B-Binary (Dummy)
#P-Percentage change
#R-Rate of change
#D-Discreet--qualitative
#Z-Z Score
#T-Time

#Create a new dataset of jump observations
###Categories###
#*0: Company Name/Ticker Symbol
#1:S Size of company (Market cap pre- and post-jump)
#*2:B Direction (+/-, good/bad news)
#*3:S Time: Does this relationship between news and jumps change in different market conditions?
#4:B Period (Does the relationship between news and jumps change in different market conditions (recession, recovery))
###Possibly just use S&P 500 or define broad market categories
#*4:P Magnitude (percent change in price from before jump until after jump)
#5:R Average Rate of change (magnitude/time [measured in days])
#6:D Industry (do different industries behave differently)
#7:B Sustainability-do prices return to pre-jump level (separate test)
#8: Accompanying Google Trends pattern
###Increased interest before/after/during
###sustained interest sustained increased interest?
###Magnitude
###Rate of change
#*9:Z News volatility (maybe find a relationship between this and company size)
#10:Z Stock Price Volatility
#10:T Duration: when do prices stabilize?

#Defining jump: like Robot Broker Program, but with variable x time and variable percentage p
#get rid of redundancy and characterize different types of movement

In [ ]:
stockHistory=pd.read_csv('WilshireHist')

In [ ]:
#graph=stockHistory.plot.line()
#graph.draw()

In [ ]:
stockHistory

In [ ]:
jumpFilter=pd.DataFrame()
#jumpFilter['ticker']=[]
#jumpFilter['percentjumpy']=[]
#jumpFilter['posneg']=[]
#jumpFilter['startdate']=[]


In [ ]:
jumpFilter

In [ ]:
print stockHistory.iloc[0,0]

In [ ]:
type(stockHistory.values[1][0])

In [ ]:
def jumpFilter(df, y=.1,x=1):
    lastPrice=1000 #something really high so that we do not have to call the first thing a jump
    listOfJumps=[] #ticker, date, jump threshold passed, lookahead period, pos/neg, percentJump, avgdailyrateofchange, counter
    y=.1 #threshold
    x=1 #lookahead period
    for i in range(len(df.columns[1])):
        counter=0
        for j in range(len(df.values)):
            if (j+x)<len(df.values):
                if type(df.values[j][i])!=str:
                    if math.isnan(float(df.values[j][i]))==False:
                        currentPrice=df.values[j][i]
                        futurePrice=df.values[j+x][i]
                    if futurePrice>currentPrice:
                        posneg=1
                    elif futurePrice<currentPrice:
                        posneg=0
                    percentJump=abs((currentPrice-futurePrice)/currentPrice)
                    if (currentPrice-futurePrice)/currentPrice>=y:
                        counter+=1
                        avgDailyRateChange=percentJump/(x+counter-1)
                    elif (currentPrice-futurePrice)/currentPrice<y and counter>=1:
                        listOfJumps.append([df.columns[i],df.values[j][0], y, x, posneg, percentJump, avgDailyRateChange, counter])
                        counter=0
print jumpFilter(stockHistory)

In [34]:
bonk=float('NaN')
bonk
math.isnan(bonk)
type(bonk)

float

In [11]:
#jumplist=[]
#for rowIndex in range(len(stockHistory.values)): #delete the 3 for enlarged application
#    for ticker in range(len(stockHistory.columns[1:3])):
#        myfavthings.append(stockHistory.iloc[ticker,rowIndex])
#jumplist

In [12]:
#jumpFilter['stuff']=jumplist

In [13]:
#stockHistory.values

In [14]:
#list(stockHistory)

## Companies we had in mind:

Dryships
<img src='DRYS.png'>
(Source: Yahoo Finance)

Semi-LED Corp
<img src='DRYS.png'>
(Source: Yahoo Finance)

Chipotle
<img src='CMG.png'>
(Source: Yahoo Finance)

In [17]:
#read in the csv with the russell 3000 stock data and Wilshire 5000 stock data (Just a dataframe of symbols for now)
#Commented out since the repository now contains this csv

dfRussell=pd.read_csv('russell_3000_2011-06-27.csv')
dfWilshire=pd.read_csv('wilshire5000.csv')

In [18]:
#Taking the values from the dataframes and making them into arrays
#Commented out since the repository now contains this csv

arrayRussell=dfRussell.values.flatten()
arrayWilshire=dfWilshire.values.flatten()

In [ ]:
#Make the dataframe with the adjusted close prices for all of the Wilshire 5000 Companies
#Commented out since the repository now contains this csv

ls_key = 'Adj Close'
start = datetime.datetime(2004,1,1)
end = datetime.datetime(2016,12,31)
f = data.DataReader(arrayWilshire[:], 'yahoo',start,end)

cleanData = f.ix[ls_key]
dataFrame = pd.DataFrame(cleanData)

print dataFrame[:]

/Users/grotero/anaconda/lib/python2.7/site-packages/pandas_datareader/base.py:174: SymbolWarning: Failed to read symbol: 'AAII', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/grotero/anaconda/lib/python2.7/site-packages/pandas_datareader/base.py:174: SymbolWarning: Failed to read symbol: 'AATI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/grotero/anaconda/lib/python2.7/site-packages/pandas_datareader/base.py:174: SymbolWarning: Failed to read symbol: 'ABAT', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/grotero/anaconda/lib/python2.7/site-packages/pandas_datareader/base.py:174: SymbolWarning: Failed to read symbol: 'ABBC', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/grotero/anaconda/lib/python2.7/site-packages/pandas_datareader/base.py:174: SymbolWarning: Failed to read symbol: 'ABD', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/Users/grotero/a

In [ ]:
#Make the dataframe into a csv for turning into more meaningful data on 'jumps'
#Commented out since the repository now contains this csv

dataFrame.to_csv('WilshireHist')

In [ ]:
yahoo=Share('YHOO')

In [ ]:
yahoo.get_market_cap()

# Land of Forgotten Code

In [8]:
#start=datetime.datetime(2013,1,1)
#end=datetime.datetime(2017,1,1)
#df = data.DataReader(arrayWilshire[0:4], 'yahoo', start, end) 
#print df
#dates =[]
#for x in range(len(df)):
#    newdate = str(df.index[x])
#    newdate = newdate[0:10]
#    dates.append(newdate)

#df['dates'] = dates

#print df.head()
#print df.tail()

In [ ]:
#yahoo=Share('YHOO')
#print yahoo
#yahoo.get_price()

#stocklist = ['aapl','goog','fb','amzn','COP']

#http://www.jarloo.com/yahoo_finance/
#https://greenido.wordpress.com/2009/12/22/yahoo-finance-hidden-api/
#_yahoo_codes.update({'Market Cap': 'j1'})
#_yahoo_codes.update({'Div Yield': 'y'})
#_yahoo_codes.update({'Bid': 'b'})
#_yahoo_codes.update({'Ask': 'a'})
#_yahoo_codes.update({'Prev Close': 'p'})
#_yahoo_codes.update({'Open': 'o'})
#_yahoo_codes.update({'1 yr Target Price': 't8'})
#_yahoo_codes.update({'Earnings/Share': 'e'})
#_yahoo_codes.update({"Day’s Range": 'm'})
#_yahoo_codes.update({'52-week Range': 'w'})
#_yahoo_codes.update({'Volume': 'v'})
#_yahoo_codes.update({'Avg Daily Volume': 'a2'})
#_yahoo_codes.update({'EPS Est Current Year': 'e7'})
#_yahoo_codes.update({'EPS Est Next Quarter': 'e9'})

#data.get_quote_yahoo(stocklist).to_csv('test.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

#data.get_quote_yahoo(stocklist).transpose()